# Snowflake Cortex AI - Transforming & Semantic Model Generation

This notebook shows how you can use Large Language Models to help you transforming from raw, bronze to gold layer with the goal of generating a semantic model where users can ask questions using natural language. The bronze layer will have tables with column names that meaningful to Analyst. The Gold Layer will have the views that will be used by Cortex Analyst, Snowflake text-2-sql capability. 

In [ ]:
# Import python packages
import streamlit as st
import pandas as pd

# We can also use Snowpark for our analyses!
from snowflake.snowpark.context import get_active_session
session = get_active_session()
session

Change this definition in case you want to use other names or LLMs. Note this has been tested using the Anthropic model claude-3-5-sonnet. For LLM Region availability check: https://docs.snowflake.com/en/user-guide/snowflake-cortex/llm-functions or enable Cross-Region Support.

In [ ]:
LLM = 'claude-3-5-sonnet'
RAW_LAYER = 'SAP_RAW'
BRONZE_LAYER = 'SAP_BRONZE'
GOLD_LAYER = 'SAP_GOLD'


Note this Notebook is expected to run in a test database and it will create a clean environment. This is replacing the BRONZE_LAYER schema in case it exists!!

In [ ]:
session.sql(f'create or replace schema {BRONZE_LAYER}').collect()

### Asking LLM how to generate new tables in BRONZE layer and copy from RAW

Let's use the power of LLMs to translate column names into something meaningful for analyst and provide the SQL to crate new tables and copy the content from RAW ones. This will take a few minutes

In [ ]:
import re

table_names = ["0fi_ar_4", "0customer_attr", "0material_attr"]

results = {}

for table in table_names:
    
    sql_text = f"""
    select snowflake.cortex.complete('{LLM}', '
    Generate a new SQL CREATE OR REPLACE table statement which will replace all 
    the below column names by easy and clear to understand column names for a 
    data analyst. Generate the SQL to copy the data from the source table to the
    target table respecting the column names. 
    The data types should be kept the same. Source schema is called {RAW_LAYER}. 
    Target schema is called {BRONZE_LAYER}. 
    It should in run Snowflake.'  
    || GET_DDL('table','{RAW_LAYER}."{table}"') );
    """

    full_text_str = session.sql(sql_text).collect()[0][0]

    # Extract content between ```sql and ```
    match = re.search(r"```sql(.*?)```", full_text_str, re.DOTALL)
    if match:
        extracted_sql = match.group(1).strip()
        extracted_sql = extracted_sql.replace('"', '\"')  # Double double-quotes for SQL safety

    else:
        extracted_sql = '""" """'
        
    results[table] = {
        "table_name": table,
        "full_output": full_text_str,
        "extracted_sql": extracted_sql
    }



Visualize the results before executing. Click on each expander to review:

In [ ]:
for table, data in results.items():
    with st.expander(f"Full Output for Table: {data['table_name']}"):
        st.subheader("Full Output:")
        st.code(data["full_output"], language="sql")

    with st.expander(f"SQL for Table: {data['table_name']}"):
        st.subheader("Extracted SQL:")
        st.code(data["extracted_sql"], language="sql")

If you are ok with the output, run next cell to extract the SQL provided to CREATE the new table and the one to INSERT the data:

In [ ]:
import re

create_sql_statements = ""

for table, data in results.items():
    extracted_sql = data["extracted_sql"]

    # Updated regex to ensure we capture full CREATE and INSERT statements ending in ');'
    sql_statements = re.split(r';\s*\n', extracted_sql.strip())
    
    # Extracting the CREATE TABLE and INSERT statements
    create_table_sql = sql_statements[0] + ";"
    insert_sql = sql_statements[1] + ";"
    
    print(f"CREATING TABLE: {table}")
    session.sql(create_table_sql).collect()
    create_sql_statements += create_table_sql
    
    print(f"INSERT INTO TABLE {table}:")
    session.sql(insert_sql).collect()



One of the advantages of Snowflake Notebooks is that you can combine Python, SQL and Markdown! Let's see what tables we got in this layer:

In [ ]:
show tables;

In [ ]:
select * from ACCOUNTS_RECEIVABLE limit 5;

### Generaing GOLD Layer

Let's use the power of LLMs to create a Data MART with the previous tables. This will contain the most important information joining the 3 tables:


In [ ]:
sql_text3 = f"""
select snowflake.cortex.complete('{LLM}', '
Generate a Data MART table selecting up to 20 most representative columns and joining they key columns properly.
Create the new table under {GOLD_LAYER} schema.
These are the tables to be used:
{create_sql_statements}');
"""
#print (sql_text)

full_text_str2 = session.sql(sql_text3).collect()[0][0]

# Extract content between ```sql and ```
match = re.search(r"```sql(.*?)```", full_text_str2, re.DOTALL)
if match:
    extracted_sql2 = match.group(1).strip()
    extracted_sql2 = extracted_sql2.replace('"', '\"')  # Double double-quotes for SQL safety

else:
    extracted_sql2 = '""" """'
    

Check results:

In [ ]:
with st.expander(f"Full Explanation:"):
    st.code(full_text_str2, language="sql")

with st.expander(f"SQL to build Mart:"):
    st.code(extracted_sql2, language="sql")

Create the GOLD schema and crate the table:

In [ ]:
session.sql('create or replace schema SAP_GOLD').collect()
session.sql(extracted_sql2).collect()

In [ ]:

select * from ACCOUNTS_RECEIVABLE_MART limit 50;



### Semantic Model: Cortex Search and Cortex Analyst

Using Snowflake Snowsight UI, we are going to create the Semantic Model that will be used by Cortex Analys to allow business users ask questions in natural language.

As we have a large number of CUSTOMER_NAME and CUSTOEMR_CITY, and Aanlyst may want to ask for any of them, we are going to use the integration of Cortex Search and Cortex Analyst. 

Cortex Search will be enabled on those columns, so Cortex Analyst can retrieval names when needed, without having to provide all possible names in the semantic file.

In [ ]:

CREATE OR REPLACE CORTEX SEARCH SERVICE _ANALYST_CUSTOMER_NAME_SEARCH
  ON CUSTOMER_NAME
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT CUSTOMER_NAME
  FROM ACCOUNTS_RECEIVABLE_MART
);

CREATE OR REPLACE CORTEX SEARCH SERVICE _ANALYST_CUSTOMER_CITY_SEARCH
  ON CUSTOMER_CITY
  WAREHOUSE = COMPUTE_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
AS (
  SELECT
      DISTINCT CUSTOMER_CITY
  FROM ACCOUNTS_RECEIVABLE_MART
);



Let's crate on Stage to keep the Semantic Models that will be crated later using the Cortex Analyst Semantic Model Generator.

In [ ]:
use schema PUBLIC;

CREATE OR REPLACE STAGE SEMANTIC_MODELS
  DIRECTORY = (ENABLE = TRUE)
  ENCRYPTION = ( TYPE = 'SNOWFLAKE_SSE' );

  

In [ ]:
sql_text = f"""
select snowflake.cortex.complete('{LLM}', '
Provide 5 examples of questions that can be asked to this data mart:
{extracted_sql2}');
"""
#print (sql_text)

full_text_str3 = session.sql(sql_text).collect()[0][0]


In [ ]:
with st.expander("Questions:"):
    st.code(full_text_str3, language="sql")